In [5]:
from openai import AzureOpenAI
from openai import OpenAI

In [6]:
# api_version = "2024-07-18"
api_version = "2024-02-15-preview"
azure_endpoint="https://nshan-m3oer8jc-eastus.openai.azure.com/"
azureClient = AzureOpenAI(api_version=api_version, azure_endpoint=azure_endpoint)
# azureClient = AzureOpenAI(azure_endpoint=azure_endpoint)

In [7]:
q = 'the course has already started, can I still enroll?'
response = azureClient.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": q}]
)

In [8]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started typically depends on the institution's policies, the type of course, and how far along in the term you are. Here are some general steps you can take:\n\n1. **Check with the Institution**: Contact the admissions office or check the institution's website for enrollment policies.\n2. **Speak to the Instructor**: Sometimes instructors can make exceptions for late enrollments.\n3. **Assess Course Requirements**: Understand what you may miss if you join late, as participation and materials may be key to keeping up.\n4. **Look for Additional Options**: Some institutions offer online or accelerated versions of courses that you may be able to join.\n\nIt's always best to inquire directly for the most accurate information."

In [12]:
from langchain.document_loaders import PyPDFLoader

# Load the book
loader = PyPDFLoader("https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf")
pages = loader.load_and_split()

### Pre processing

Remove non printable char & extra spaces

In [13]:
# Cut out the open and closing parts
pages = pages[6:1308]
# Combine the pages, and replace the tabs with spaces
text = ' '.join([page.page_content.replace('\t', ' ') for page in pages])

In [14]:
text[0:200]

'7\nThe MLOps lifecycle\nThe MLOps lifecycle encompasses seven integrated and iterative processes, as shown in figure 2.\nThe processes can consist of the following:\n• ML development concerns experimentin'

In [15]:
import re
def clean_text(text):
   # Remove extra spaces
   cleaned_text = re.sub(r' +', ' ', text)
   # Remove non-printable characters, optionally preceded by 'David Copperfield'
   cleaned_text = re.sub(r'[\x00-\x1F]', '', cleaned_text)
   # Replace newline characters with spaces
   cleaned_text = cleaned_text.replace('\n', ' ')
   # Remove spaces around hyphens
   cleaned_text = re.sub(r'\s*-\s*', '', cleaned_text)
   return cleaned_text

In [16]:
clean_text=clean_text(text)

In [1]:
from langchain_openai import AzureOpenAI

In [2]:
llm = AzureOpenAI(
    model='gpt-4o-mini'
)

In [4]:
llm.invoke("Hello, world!")

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

In [19]:
llm.get_num_tokens(clean_text)

9283

Split doc using SemanticChunker

In [35]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
text_splitter = SemanticChunker(
   OpenAIEmbeddings(), breakpoint_threshold_type="interquartile"
)
docs = text_splitter.create_documents([clean_text])

In [37]:
import numpy as np
import openai
def get_embeddings(text):
   response = AzureOpenAI.embeddings.create(
       model="text-embedding-3-small",
       input=text
   )
   return response.data
embeddings=get_embeddings([doc.page_content for doc in docs]
)

AttributeError: embeddings